In [109]:
import pandas as pd
import re
try:
    with open("claves.sdf", "r") as archivo:
        df_claves = pd.DataFrame(columns=['lito_clave', 'tema_clave', 'solucion'])
        data_claves = archivo.readlines()
        for line in data_claves:
            lito = line[:6]
            tema = line[6]
            solucion = line[7:].replace('\n', '')
            df_claves = pd.concat([df_claves, pd.DataFrame({'lito_clave': [lito], 'tema_clave': [tema], 'solucion': [solucion]})], ignore_index=True)            
    print(df_claves)
except Exception as e:
    print('Hubo un error: ', e)
try:
    with open("i.sdf", "r") as archivo:
        df_identifi = pd.DataFrame(columns=['lito', 'tema', 'codigo'])
        data_identifi = archivo.readlines()
        for linea in data_identifi:
            lito = linea[:6]
            tema = linea[6]
            codigo = linea[7:].replace('\n', '')
            df_identifi = pd.concat([df_identifi, pd.DataFrame({'lito': [lito], 'tema': [tema], 'codigo': [codigo]})], ignore_index=True)
    print(df_identifi)
except Exception as e:
    print('Hubo un error: ', e)
try:
    with open("r.sdf", "r") as archivo:
        df_respuestas = pd.DataFrame(columns=['lito', 'tema', 'respuesta'])
        data_respuestas = archivo.readlines()
        for linea in data_respuestas:
            lito = linea[:6]
            tema = linea[6]
            respuesta = linea[7:].replace('\n', '')
            df_respuestas = pd.concat([df_respuestas, pd.DataFrame({'lito': [lito], 'tema': [tema], 'respuesta': [respuesta]})], ignore_index=True)
    print(df_respuestas)
except Exception as e:
    print('Hubo un error: ', e)

  lito_clave tema_clave                                           solucion
0     033872          A  TRRQRPRTRRQSTQTQPQQRQSQRQSSRTSTPQRTRPSTRRQQTRS...
1     033871          B  TTPTRTPTSSSTPTTSPPSQRRTQPRPRSPTTRTQQPPSPP STRR...
2     033874          C  PQQSQRQTTQRSRPPRPQTPRPPTSTPRSQQTTSPTSQTTTTRSQQ...
3     033873          D  PTRPRTRSTSSTSPSTPTQSRSQQQSQRQSRPSQPPRTRPRRTPPS...
4     033880          P  TPPRRTRSQSRQQSPRPRQRSQQTPTSSPTTPTTPRRQSSPQQSRQ...
5     033877          Q  PTRQPTPTSRQQQQQPSQQRPSRQSQTPPSPQPTRRPPQTTQQRRP...
6     033876          R  QPTSPSTSSTTQSTPQRSPSQTRPTRTPPSSQSRTTPPTTTPTTQS...
7     033875          S  QTRTSQSPSRRPSPRPPTRRRPPRTTRQRPSSSPTPQRQPQSSPPS...
     lito tema  codigo
0  010486    A  350025
1  010515    R  306126
2  010489    C  326710
3  010498    S  320804
4  010492    B  320802
     lito tema                                          respuesta
0  010486    A  SR STQRRST R  P QSTQRPPTRTP RPQPTS SSTQSR R  Q...
1  010515    R  PQQPRQSPTSRSQRQRPTQPTTQPQTRRQ RPSPPPST 

### Validar duplicado codigo

In [89]:
def duplicated_code_solution(df_identifi):
    res = ""

    # cambia el indice
    df_identifi = df_identifi.reset_index(drop=True)
    df_identifi.index += 1

    duplicados = df_identifi.duplicated(subset=['codigo'], keep=False)
    if duplicados.any():
        res += "Se encontraron codigos duplicados:\n\n"
        datos_duplicados = {}
        for codigo, df_codigo in df_identifi[duplicados].groupby('codigo'):
            if len(df_codigo) > 1:
                datos_duplicados[codigo] = {'lito': [], 'tema': [], 'indices': []}
                for idx, fila in enumerate(df_codigo.itertuples(), start=1):
                    datos_duplicados[codigo]['lito'].append(fila.lito)
                    datos_duplicados[codigo]['tema'].append(fila.tema)
                    datos_duplicados[codigo]['indices'].append(fila.Index)
                df_duplicados = pd.DataFrame({'lito': datos_duplicados[codigo]['lito'], 'tema': datos_duplicados[codigo]['tema'], 'codigo': [codigo]*len(df_codigo), 'fila': datos_duplicados[codigo]['indices']})
                df_duplicados = df_duplicados[['lito', 'tema', 'codigo', 'fila']]
                res += f"{str(df_duplicados)}\n"
    else:
        res += "No se encontraron duplicados en la columna 'codigo'"
    return res

sol = duplicated_code_solution(df_identifi)
print(sol)

No se encontraron duplicados en la columna 'codigo'


### VALIDAR DUPLICADO DE LITO DE DF_IDENTIFI Y DF_RESPUESTA 

In [88]:
def duplicated_litio_identifi(df_identifi):
    # cambia el indice
    df_identifi = df_identifi.reset_index(drop=True)
    df_identifi.index += 1

    # Verifica si hay duplicados en la columna 'lito'
    duplicados = df_identifi.duplicated(subset=['lito'], keep=False)

    # Filtra las filas que contienen duplicados
    filas_con_duplicados = df_identifi.where(duplicados).dropna()
    

    if filas_con_duplicados.empty:
        return "No hay duplicados"
    else:
        return filas_con_duplicados
def duplicated_litio_solution(df_respuestas):
    # cambia el indice
    df_respuestas = df_respuestas.reset_index(drop=True)
    df_respuestas.index += 1

    # Verifica si hay duplicados en la columna 'lito'
    duplicados = df_respuestas.duplicated(subset=['lito'], keep=False)

    # Filtra las filas que contienen duplicados
    filas_con_duplicados = df_respuestas.where(duplicados).dropna()
    

    if filas_con_duplicados.empty:
        return "No hay duplicados"
    else:
        return filas_con_duplicados

print(f"Duplicado litio identificador\n{duplicated_litio_identifi(df_identifi)}\n")
print(f"Duplicado litio respuesta\n{duplicated_litio_solution(df_respuestas)}\n")

Duplicado litio identificador
     lito tema  codigo
4  010498    S  320804
5  010498    B  320802

Duplicado litio respuesta
No hay duplicados



### VALIDAR LITO NO LOCALIZADO

In [82]:
# def sin_pareja(df_id, df_resp):
#     ser_lit_id = pd.Series(df_id['lito'], index=range(len(df_id['lito'])))
#     ser_lit_re = pd.Series(df_resp['lito'], index=range(len(df_resp['lito'])))
#     ser_merge = ser_lit_id.isin(ser_lit_re)
#     no_pareja = 'orden | litho  | tema | codigo \n'
#     for i in range(len(df_id)):
#         if not ser_merge[i]:
#             no_pareja += f'{i+1}  | {df_id.iloc[i,0]} |  {df_id.iloc[i,1]}   | {df_id.iloc[i,2]} \n'
#         else:
#             no_pareja = 'Todos los litos estan localizados'
#     return no_pareja
# print(sin_pareja(df_identifi, df_respuestas))

In [95]:
# Unir los dataframes y filtrar las filas no localizadas
merged = pd.merge(df_identifi, df_respuestas, on=['lito', 'tema'], how='outer', indicator=True)
no_localizados = merged[merged['_merge'] == 'left_only']

# Mostrar los resultados
print('Dataframe no localizado')
print(no_localizados)

Dataframe no localizado
     lito tema  codigo respuesta     _merge
2  010489    C  326710       NaN  left_only
4  010498    B  320802       NaN  left_only


In [116]:
# Unir los dataframes y filtrar las filas no localizadas
merged = pd.merge(df_identifi, df_respuestas, on=['lito', 'tema'], how='outer', indicator=True)
no_localizados = merged.query("_merge == 'right_only'").loc[:, ['lito']]

# Mostrar los resultados
print('Dataframe no localizado')
print(pd.DataFrame({'lito': no_localizados['lito']}))

Dataframe no localizado
     lito
5  010484
6  010512


In [113]:
# Unir los dataframes y filtrar las filas no localizadas
merged = pd.merge(df_respuestas, df_identifi, on=['lito', 'tema'], how='outer', indicator=True)
no_localizados = merged.query("_merge == 'right_only'").loc[:, ['lito']]

# Mostrar los resultados
print('Dataframe no localizado')
print(pd.DataFrame({'lito': no_localizados['lito']}))

Dataframe no localizado
     lito
5  010489
6  010492
